# MODEL TRAINING - testing the full approach
saving models in a different folder using pickle, saving them in the dataframe causes xgboost to crash

**DO NOT RUN AGAIN**

In [1]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"/COGNITION"
dreamspath=datapath+"/DREAMS"
masspath=datapath+"/MASS"


## experiment id
we'll use it to create a folder at the end of the script

In [2]:
experimentId=str(uuid.uuid4())

## define a fixed samplerate

In [3]:
samplerate=200

## load mass

In [4]:
#annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate,onlySpindlesFilteredN2=True)
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [5]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

33458

In [6]:
minDuration=0.3
maxDuration=5
annotations=annotations[annotations.duration>minDuration].reset_index(drop=True)
annotations=annotations[annotations.duration<maxDuration].reset_index(drop=True)
len(annotations)

33269

In [7]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [8]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## define a fixed feature selection to use

In [9]:
featureSelection=loadPickle("featureSelection26.pkl")
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## load data split
5 fold CV, 3 validation subjects

In [11]:
#by now we used fixed CV data splits instead
dataSplits=loadPickle("dataSplits_5Fold.pkl")
dataSplits

,train,val,test
0,"[0001, 0005, 0006, 0009, 0010, 0011, 0012, 001...","[0018, 0017, 0007]","[0013, 0002, 0003]"
1,"[0002, 0003, 0006, 0007, 0009, 0013, 0017, 001...","[0011, 0001, 0010]","[0005, 0014, 0012]"
2,"[0003, 0005, 0007, 0009, 0010, 0012, 0013, 001...","[0001, 0002, 0019]","[0011, 0006, 0018]"
3,"[0002, 0003, 0005, 0006, 0007, 0012, 0013, 001...","[0009, 0011, 0017]","[0019, 0001, 0010]"
4,"[0001, 0003, 0005, 0006, 0010, 0011, 0013, 001...","[0014, 0012, 0002]","[0009, 0017, 0007]"


## define annotation criteria
we want to test the models in three diferent ways

In [12]:
annotationCriteria=pd.DataFrame({
    'criteriumId':['0001','0002'],
    'criteriumName':['E1','E2'],
    'labelerIdList':[['0001'],['0002']]
})

In [13]:
annotationCriteria

,criteriumId,criteriumName,labelerIdList
0,0001,E1,[0001]
1,0002,E2,[0002]


In [14]:
experimentModels=pd.merge(annotationCriteria,dataSplits,how='cross')

In [15]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test
0,0001,E1,[0001],"[0001, 0005, 0006, 0009, 0010, 0011, 0012, 001...","[0018, 0017, 0007]","[0013, 0002, 0003]"
1,0001,E1,[0001],"[0002, 0003, 0006, 0007, 0009, 0013, 0017, 001...","[0011, 0001, 0010]","[0005, 0014, 0012]"
2,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0012, 0013, 001...","[0001, 0002, 0019]","[0011, 0006, 0018]"
3,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0012, 0013, 001...","[0009, 0011, 0017]","[0019, 0001, 0010]"
4,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0010, 0011, 0013, 001...","[0014, 0012, 0002]","[0009, 0017, 0007]"
5,0002,E2,[0002],"[0001, 0005, 0006, 0009, 0010, 0011, 0012, 001...","[0018, 0017, 0007]","[0013, 0002, 0003]"
6,0002,E2,[0002],"[0002, 0003, 0006, 0007, 0009, 0013, 0017, 001...","[0011, 0001, 0010]","[0005, 0014, 0012]"
7,0002,E2,[0002],"[0003, 0005, 0007, 0009, 0010, 0012, 0013, 001...","[0001, 0002, 0019]","[0011, 0006, 0018]"
8,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0007, 0012, 0013, 001...","[0009, 0011, 0017]","[0019, 0001, 0010]"
9,0002,E2,[0002],"[0001, 0003, 0005, 0006, 0010, 0011, 0013, 001...","[0014, 0012, 0002]","[0009, 0017, 0007]"


## model fitting

In [16]:
n_jobs=4 # paralelization parameter for xgboost (keep it 8 as maximum)
learning_rate=0.4
subsample=0.6

num_boost_round=60  #number of boosting rounds per model

models=[]
modelIds=[]
valF1s=[]
spindleTimeRates=[]
for index, row in experimentModels.iterrows():
    print('**********************')
    print(str(index+1)+' of '+str(len(experimentModels)))
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Features
    trainFeatures=loadFeatureMatrix(row.train,featureSelection,signalsMetadata,samplerate,datapath)
    #Labels
    trainLabels=loadLabelsVector(row.train,usedAnnotations,signalsMetadata,samplerate)

    """ # Restrict trainingto N2 --------------------------------------------------------------->
    maskVector=loadN2maskVector(row.train,hypnograms,signalsMetadata,samplerate)
    trainFeatures=trainFeatures[maskVector,:]
    trainLabels=trainLabels[maskVector]
    #<--------------------------------------------------------------------------------------- """   #not in this experiment
    
    #Train the models
    params={
        'n_jobs':n_jobs,
        'learning_rate':learning_rate,
        'subsample':subsample,
        'objective':'binary:logistic'
    }
    trainDMatrix=xgb.DMatrix(data=trainFeatures,label=trainLabels)
    xgb_model = xgb.train(params,trainDMatrix,num_boost_round=num_boost_round)
    spindleTimeRates.append(sum(trainLabels)/len(trainLabels))
    models.append(xgb_model)
    modelIds.append(str(uuid.uuid4()))
    

**********************
1 of 10
**********************
2 of 10
**********************
3 of 10
**********************
4 of 10
**********************
5 of 10
**********************
6 of 10
**********************
7 of 10
**********************
8 of 10
**********************
9 of 10
**********************
10 of 10


In [17]:
experimentModels["modelId"]=modelIds
experimentModels["spindleTimeRate"]=spindleTimeRates

In [18]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0001, 0005, 0006, 0009, 0010, 0011, 0012, 001...","[0018, 0017, 0007]","[0013, 0002, 0003]",5c1fbad9-f02b-46f7-8e51-59ddaf0cf4eb,0.017868
1,0001,E1,[0001],"[0002, 0003, 0006, 0007, 0009, 0013, 0017, 001...","[0011, 0001, 0010]","[0005, 0014, 0012]",f3f5e58f-0068-4046-b0a7-a62694b1fc26,0.017785
2,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0012, 0013, 001...","[0001, 0002, 0019]","[0011, 0006, 0018]",919e22eb-bc1e-4601-a070-09b25fe7b52f,0.017121
3,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0012, 0013, 001...","[0009, 0011, 0017]","[0019, 0001, 0010]",c2e2175e-cc0c-4536-86b5-c7a6ef28b104,0.017512
4,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0010, 0011, 0013, 001...","[0014, 0012, 0002]","[0009, 0017, 0007]",0f1d8cd5-f3dc-4419-a46b-69852cdbdadf,0.016249
5,0002,E2,[0002],"[0001, 0005, 0006, 0009, 0010, 0011, 0012, 001...","[0018, 0017, 0007]","[0013, 0002, 0003]",707edb30-2665-4c58-a31e-dbcc539eb169,0.061583
6,0002,E2,[0002],"[0002, 0003, 0006, 0007, 0009, 0013, 0017, 001...","[0011, 0001, 0010]","[0005, 0014, 0012]",b302a855-d3de-463c-b23c-53ddf80cbe46,0.053240
7,0002,E2,[0002],"[0003, 0005, 0007, 0009, 0010, 0012, 0013, 001...","[0001, 0002, 0019]","[0011, 0006, 0018]",cca39540-d753-4b1f-b2f8-6f3e209a0deb,0.056399
8,0002,E2,[0002],"[0002, 0003, 0005, 0006, 0007, 0012, 0013, 001...","[0009, 0011, 0017]","[0019, 0001, 0010]",4c8b7412-a382-4f84-b66c-acdcca60a4a9,0.051154
9,0002,E2,[0002],"[0001, 0003, 0005, 0006, 0010, 0011, 0013, 001...","[0014, 0012, 0002]","[0009, 0017, 0007]",cd965d0c-da5d-44f4-b337-5d6385102e27,0.053971


## save results

In [19]:
#create parent folder
experimentpath=datapath+"/experiments/"+experimentId
os.mkdir(experimentpath)
#save each of the models
for id, model in zip(modelIds,models):
    model.save_model(experimentpath+"/"+id+".json")
#save experiment information
dumpPickle(experimentpath+"/experimentModels.pkl",experimentModels)
dumpPickle(experimentpath+"/featureSelection.pkl",featureSelection)

In [20]:
#you need to use this in the following parts of the experiment
print(experimentId)

953846a1-fd19-4195-904d-79525add579b
